In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np


# Load and preprocess the input image
# img_path = 'cat.jpg'
img_path = 'car.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Load the VGG16 model
model = VGG16(weights='imagenet')



# Make predictions
preds = model.predict(x)
decoded_preds = decode_predictions(preds, top=3)[0]

# Print the top predictions
for _, label, prob in decoded_preds:
    print(f"{label}: {prob}")
    

1/1 [==============================] - 0s 290ms/step
convertible: 0.6561403274536133
sports_car: 0.08867453783750534
limousine: 0.0885283574461937


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image_input

# Specify the path to the dataset directory
dataset_dir = 'C:/Users/schul/OneDrive - HTW Dresden/Documents/6._Semester/Applied_AI/Code/src/week_10/Datasets'

target_size = (128, 128)


# Initialize lists to store the images and labels
images = []
labels = []
preprocess_input = []

# Iterate over the subdirectories in the dataset directory
for subdir in os.listdir(dataset_dir):
    subdir_path = os.path.join(dataset_dir, subdir)
    if os.path.isdir(subdir_path):
        # Extract the class label from the subdirectory name
        label = subdir
        # Iterate over the image files in the subdirectory
        for file_name in os.listdir(subdir_path):
            # Read the image file
            image_path = os.path.join(subdir_path, file_name)

            # try:
            labels.append(label)
            image_input = cv2.imread(image_path, cv2.COLOR_BGR2GRAY)
            # Preprocess the image (resize, normalize, etc.)
            image_input = cv2.resize(image_input, target_size)
            images.append(image_input)
            img = image_input.load_img(image_path, target_size=(224, 224))
            x = image_input.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            preprocess_input.append(preprocess_input(x))
            print("test")
            # except Exception as e:
            #     1+1

# Convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels)
print(preprocess_input)